# Preparing NISAR-like data for validation of Solid Earth requirements

Prepared by Ekaterina Tymofyeyeva, Heresh Fattahi, Sara Mirzaee, Max Zhan, and Jeff Pon March 2024, 

<div class="alert alert-warning">
Both the initial setup (<b>Prep A</b> section) and download of the data (<b>Prep B</b> section) should be run at the start of the notebook. Methods for validation of transient, secular, and coseismic requirements using Sentinel-1 ARIA data can be run subsequently.
</div>

<hr/>



In [ ]:
## Define CalVal Site 
# This is a NISAR-like test dataset from California derived from ALOS-1

site='NISAR_ALOS1' 

# Specify the directory where you want to do all your work and store your output time series
my_directory = '/scratch/katia/ATBD_outputs'

# Where the raw data are stored (staged for now, later searchable and streamed directly from the DAAC)
gunw_dir = '/scratch/katia/ALOS_ROSAMOND_20242610'

# Environment Setup
Setup your environment for processing data

In [ ]:
#Load Packages
import glob
import os
import subprocess
from datetime import datetime as dt
from pathlib import Path

import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
#from mintpy import view, plot_network
from mintpy.cli import view, plot_network
from mintpy.objects import gps, timeseries
from mintpy.smallbaselineApp import TimeSeriesAnalysis
from mintpy.utils import ptime, readfile, utils as ut
from scipy import signal

from solid_utils.sampling import load_geo, samp_pair, profile_samples, haversine_distance

#Set Global Plot Parameters
plt.rcParams.update({'font.size': 12})

################# Set Directories ##########################################

# Where the raw data are stored
data_dir = '/scratch/katia/NISAR_20240321'

# Bounding box
bbox='-119.0 33.0 -117.0 35.0'

print("   GUNW    dir:", gunw_dir) 

# Directory for MintPy processing
mintpy_dir = os.path.join(my_directory,site,'MintPy')
os.makedirs(mintpy_dir,exist_ok=True)

print("   MintPy  dir:", mintpy_dir)

### Change to MintPy workdir
vel_file = os.path.join(mintpy_dir, 'velocity.h5')
msk_file = os.path.join(mintpy_dir, 'maskConnComp.h5')  # maskTempCoh.h5 maskConnComp.h5

os.chdir(mintpy_dir)

<a id='secular_prep_b'></a>
## Set up site parameters

In [ ]:
sites = {
    ##########  NISAR_ALOS1 ##############
    'NISAR_ALOS1' : {'calval_location' : 'NISAR_ALOS1',
            'reference_lalo' : 'auto',
            'download_start_date' : '20071210',
            'download_end_date' : '20110320',
            'analysis_region' : ' 33.0 35.0 -119.0 -117.0',
            'tempBaseMax' : 'auto',
            'ifgExcludeList' : 'auto',
            'maskWater' : False}} 

print(sites[site]['analysis_region'])

S = sites[site]['analysis_region'].strip().split(' ')[0]
N = sites[site]['analysis_region'].strip().split(' ')[1]
W = sites[site]['analysis_region'].strip().split(' ')[2]
E = sites[site]['analysis_region'].strip().split(' ')[3]
bbox = W + ' ' + S + ' ' + E + ' ' + N
print('bbox: ', bbox)

<a id='secular_gen_ifg'></a>
# 1. Data access and prep
### <font color='red'>Under Construction</font> 

## 1.1 Search for data on the DAAC 

Currently, the data are staged locally, but we will be exercising searching for them on the DAAC using bounding boxes and time spans of interest.

## 1.2 Create additional interferograms to supplement your network 

- Define your network by specifying how many interferograms you want to make (n+1, n+2?)
- Optionally, recreate the entire network (including nearest neighbor pairs) using custom user-defined parameters. These can include:
    - Resampling, 
    - Stitching together multiple frames at the RSLC level,
    - Changing the unwrapping algorithm and other processing options
- Stage the additional products on the DAAC, where they will be searchable.

<a id='secular_gen_ts'></a>
# 2. Making the NISAR data cube

InSAR time series (i.e., the unfiltered displacement of each pixel vs. time) are estimated from a processed InSAR stack from Section 3.1 (either ascending or descending) using a variant of the small baseline subset (SBAS) approach and then parameterized using the approach described in Section 4. This step uses tools available in the MintPy software package (REF), which provides both SBAS time series and model-based time series parameterization. Recent results on InSAR closure phase and “fading signal” recommend the of use all available interferograms to avoid systematic bias (_Ansari et al._, 2020; _Zheng Y.J. et al._, 2022). As we expect high-quality orbital control for NISAR, we anticipate that the interferogram stack will typically include all nearest-neighbor (i.e., ~12-day pairs) and skip-1 interferograms, which will be the minimum inputs into the SBAS generation step.

We use the open-source ARIA-tools package to download processed L2 interferograms over selected cal/val regions from the Alaska Satellite Facility archive and to stitch/crop the frame-based NISAR GUNW products to stacks that can be directly ingested into MintPy for time-series processing. ARIA-tools uses a phase-minimization approach in the product overlap region to stitch the unwrapped and ionospheric phase, a mosaicing approach for coherence and amplitude, and extracts the geometric information from the 3D data cubes through a mosaicking of the 3D datacubes and subsequent intersection with a DEM. ARIA has been used to pre-process NISAR beta products derived from Sentinel-1 which have revealed interseismic deformation and creep along the San Andreas Fault system, along with subsidence, landsliding, and other signals. 

We use MintPy to validate and modify the InSAR stack, removing interferograms that do not meet minimum coherence criteria, generating a quality control mask for the purpose of identifying noisy pixels within the stack, and referencing estimated deformation to a common location in all interferograms.

<a id='secular_setup_config'></a>
## 2.1. Set Up MintPy Configuration file


The default processing parameters for MintPy's **smallbaselineApp.py** need to be modified by including the following lines in config_file (which must be manually created and placed into mint_dir):

- mintpy.load.processor      = aria
- mintpy.load.unwFile        = ../stack/unwrapStack.vrt
- mintpy.load.corFile        = ../stack/cohStack.vrt
- mintpy.load.connCompFile   = ../stack/connCompStack.vrt
- mintpy.load.demFile        = ../DEM/SRTM_3arcsec.dem
- mintpy.load.incAngleFile   = ../incidenceAngle/{download_start_date}_{download_edn_date}.vrt
- mintpy.load.azAngleFile    = ../azimuthAngle/{download_start_date}_{download_edn_date}.vrt
- mintpy.load.waterMaskFile  = ../mask/watermask.msk
- mintpy.reference.lalo      = auto, or somewhere in your bounding box
- mintpy.topographicResidual.pixelwiseGeometry = no
- mintpy.troposphericDelay.method              = no
- mintpy.topographicResidual                   = no

In [ ]:
config_file = Path(mintpy_dir)/(sites[site]['calval_location'] + '.cfg')


####################################################################
### Write smallbaseline.py config file
# TODO fix 
config_file_content = """
mintpy.load.processor = nisar
mintpy.compute.numWorker = auto
mintpy.load.unwFile = {workdir}/products/*h5
mintpy.load.demFile = {mintpydir}/DEM/elevation.dem
mintpy.topographicResidual.pixelwiseGeometry = no
mintpy.troposphericDelay.method = no
mintpy.topographicResidual = no
mintpy.network.tempBaseMax = {tempmax}
mintpy.network.startDate = {startdatenet}
mintpy.network.endDate = {enddatenet}
mintpy.velocity.startDate = {startdatevel}
mintpy.velocity.endDate = {enddatevel}
mintpy.reference.lalo = {reference_lalo}
mintpy.network.excludeIfgIndex = {excludeIfg}""".format(workdir = work_dir,
                                                        mintpydir = mintpy_dir,
                                                        tempmax=sites[site]['tempBaseMax'],
                                                        excludeIfg=sites[site]['ifgExcludeList'],
                                                        startdatenet=sites[site]['download_start_date'],
                                                        enddatenet=sites[site]['download_end_date'],
                                                        startdatevel=sites[site]['download_start_date'],
                                                        enddatevel=sites[site]['download_end_date'],
                                                        reference_lalo=sites[site]['reference_lalo'])

config_file.write_text(config_file_content)

print('MintPy config file:\n    {}:'.format(config_file))
print(config_file.read_text())

In [ ]:
# Get the DEM
dem_dir = os.path.join(mintpy_dir,'DEM')
os.makedirs(dem_dir,exist_ok=True)
demfile = os.path.join(dem_dir,'elevation.dem')

cmd = 'sardem --bbox ' + bbox + ' --data-source NASA -o ' + demfile
os.system(cmd)

<a id='secular_load_data'></a>
## 2.2. Load Data into MintPy

The output of this step is an "inputs" directory in 'calval_directory/calval_location/MintPy/" containing two HDF5 files:
- ifgramStack.h5: This file contains 6 dataset cubes (e.g. unwrapped phase, coherence, connected components etc.) and multiple metadata
- geometryGeo.h5: This file contains geometrical datasets (e.g., incidence/azimuth angle, masks, etc.)

In [ ]:
os.chdir(mintpy_dir)
print(mintpy_dir)
command = 'smallbaselineApp.py ' + str(config_file) + ' --dir ' + mintpy_dir + ' --dostep load_data'
process = subprocess.run(command, shell=True)
print('Mintpy input files:')
[x for x in os.listdir('inputs') if x.endswith('.h5')]